## Thực hiện test lấy dữ liệu và xử lý dữ liệu từ thư viện vnstock

In [1]:
!pip install ta-lib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Lấy dữ liệu từ nguồn sẵn có
import sys
import os
import talib as ta
sys.path.append('d:\\NCKH\\SARSA_FinancialRL')  # Thêm đường dẫn gốc để import module

import pandas as pd
from data.data_processor.feature_engineer import engineer_stat as es

# Định nghĩa đường dẫn gốc
base_path = 'd:\\NCKH\\SARSA_FinancialRL'

# Lấy dữ liệu tạm từ nguồn sẵn có
csv_path = os.path.join(base_path, 'data', 'data_storer', 'FPT_detail_2013_01_01_2024_12_31.csv')
df = pd.read_csv(csv_path)

In [2]:
df

,time,open,high,low,close,volume
0,20-06-2012,5.31,5.31,5.27,5.29,89720
1,21-06-2012,5.28,5.29,5.26,5.28,50360
2,22-06-2012,5.27,5.28,5.26,5.26,67350
3,25-06-2012,5.25,5.29,5.25,5.26,42840
4,26-06-2012,5.22,5.24,5.22,5.22,31110
...,...,...,...,...,...,...
3128,25-12-2024,129.85,130.37,129.68,130.02,2517969
3129,26-12-2024,130.19,130.54,128.56,129.16,2238343
3130,27-12-2024,128.90,129.25,128.21,128.99,2567003
3131,30-12-2024,128.81,130.02,128.04,129.76,1986819


In [9]:
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

# Gọi hàm với start_date và config tùy chỉnh
start_date = pd.to_datetime('01-01-2013', format='%d-%m-%Y')
df_with_indicators = es.add_technical_indicators(
    df, 
    start_date=start_date
)
# df_with_indicators giờ đã có các cột chỉ số, gắn vào df gốc của bạn
df_with_indicators

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


## Kiểm chứng độ chính xác của luồng xử lý dữ liệu và các chỉ báo kỹ thuật được thêm vào dataframe

Từ dữ liệu mẫu đã có trong biến df thực hiện tính thủ công để so sánh với kết quả tính được từ hàm đóng gói trên.

In [ ]:
# Tính thủ công các chỉ báo kỹ thuật để kiểm chứng
import numpy as np

# Sao chép df để tính thủ công
df_manual = df.copy()

# Tính MACD
df_manual['MACD'] = ta.MACD(df_manual['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]

# Tính RSI
df_manual['RSI'] = ta.RSI(df_manual['close'], timeperiod=14)

# Tính CCI
df_manual['CCI'] = ta.CCI(df_manual['high'], df_manual['low'], df_manual['close'], timeperiod=20)

# Tính ADX
df_manual['ADX'] = ta.ADX(df_manual['high'], df_manual['low'], df_manual['close'], timeperiod=14)

# Loại bỏ NaN
df_manual = df_manual.dropna().reset_index(drop=True)

# Lọc từ start_date trở đi
df_manual = df_manual[df_manual['time'] >= start_date].reset_index(drop=True)

# Định dạng time thành dd/mm/yyyy
df_manual['time'] = df_manual['time'].dt.strftime('%d/%m/%Y')

# Giữ lại các cột cần thiết
required_columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'MACD', 'RSI', 'CCI', 'ADX']
df_manual = df_manual[required_columns]

# So sánh với df_with_indicators
print("Shape df_manual:", df_manual.shape)
print("Shape df_with_indicators:", df_with_indicators.shape)

# Kiểm tra các cột giống nhau
columns_match = df_manual.columns.tolist() == df_with_indicators.columns.tolist()
print("Cột giống nhau:", columns_match)

if columns_match:
    # So sánh từng cột numeric
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'MACD', 'RSI', 'CCI', 'ADX']
    for col in numeric_cols:
        close_match = np.allclose(df_manual[col], df_with_indicators[col], rtol=1e-10, atol=1e-10)
        print(f"Cột {col} khớp:", close_match)
        if not close_match:
            print(f"  Sai lệch max trong {col}:", np.max(np.abs(df_manual[col] - df_with_indicators[col])))

    # So sánh cột time
    time_match = (df_manual['time'] == df_with_indicators['time']).all()
    print("Cột time khớp:", time_match)

print("\nĐầu df_manual:")
print(df_manual.head())

print("\nĐầu df_with_indicators:")
print(df_with_indicators.head())

Shape df ban đầu: (3133, 6)
Shape sau tính indicators: (3133, 10)
Shape sau dropna: (3100, 10)
start_date: 2013-01-01 00:00:00
Min time trong df_manual: 2012-08-06 00:00:00
Max time trong df_manual: 2024-12-31 00:00:00
Shape sau filter start_date: (2996, 10)
Shape df_manual: (2996, 10)
Shape df_with_indicators: (2996, 10)
Cột giống nhau: True
Cột open khớp: True
Cột high khớp: True
Cột low khớp: True
Cột close khớp: True
Cột volume khớp: True
Cột MACD khớp: True
Cột RSI khớp: True
Cột CCI khớp: True
Cột ADX khớp: True
Cột time khớp: True

Đầu df_manual:

Đầu df_with_indicators:


In [16]:
df_manual

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327


In [17]:
df_with_indicators

,time,open,high,low,close,volume,MACD,RSI,CCI,ADX
0,02/01/2013,4.00,4.15,4.00,4.13,380320,0.062248,72.002503,253.212693,26.054096
1,03/01/2013,4.13,4.15,4.00,4.07,366960,0.073297,66.503579,180.086580,28.047868
2,04/01/2013,4.05,4.13,4.04,4.13,278020,0.085905,69.049162,156.238566,29.899227
3,07/01/2013,4.11,4.14,4.08,4.11,204570,0.093209,67.215490,133.887463,31.676318
4,08/01/2013,4.12,4.31,4.11,4.31,535860,0.113823,74.506365,161.115250,34.114683
...,...,...,...,...,...,...,...,...,...,...
2991,25/12/2024,129.85,130.37,129.68,130.02,2517969,2.858353,67.076480,80.770363,34.110079
2992,26/12/2024,130.19,130.54,128.56,129.16,2238343,2.718885,63.212308,61.673685,33.596240
2993,27/12/2024,128.90,129.25,128.21,128.99,2567003,2.565070,62.446483,34.093919,32.865699
2994,30/12/2024,128.81,130.02,128.04,129.76,1986819,2.476752,64.541897,51.584632,32.527327
